## 資料搜集
FB 資料搜集網址 http://140.119.163.69/fbpage

粉絲ID: dpptw 1/17~1/17

(經過 Mac Numbers 再匯出成 csv 以避開 decode,encode 問題)

檔案:demo_test.csv 

## 先看一下資料

<!--使用 pandas

註：會出現 error 訊息 <span style="color:#FF0000">Skipping line 3: expected 15 fields, saw 16 </span>
所以要先增加逗點表示欄位-->

<!--f = open('demo_test.csv', 'rw')
print f.read().decode('big5').encode('utf-8')
b_str = f.read()
f.close()

print b_str.decode('big5')  # 自行判斷標準輸出編碼
print b_str.decode('utf-8').encode('big5') # 標準輸出編碼為 big5
# coding=utf-8
text = u'測試'
b_str = text.encode('big5')
f = open('text.txt', 'w')
f.write(b_str)
f.close()-->

In [1]:
f = open('demo_test.csv', 'r+')
b_str = f.read()
f.close()
print (b_str)

流水號,大類別,From,post_url,created_time,post_type,postID,post_message,status_type,link,轉貼標題(name),留言數,按讚數,分享數,update_time,
1,post,民主進步黨(150336441057),http://www.facebook.com/150336441057_10153180273481058,2016/01/19 19:20:06,link,150336441057_10153180273481058,關於「韓聯社」刊載「獨家專訪」新聞 民進黨說明      針對韓國媒體「韓聯社」於今（19）日刊載題為《[獨家]〈專訪〉蔡英文「有時間的話會看韓劇…泡菜很美味」》一文，其中包含蔡主席回答7題記者提問，內文中註明「書面專訪一問一答」，並稱「蔡英文在當選總統後，在19日接受首次接受韓國媒體訪問」。民進黨對此表示，該報導內容係引用蔡主席歷來談話，事實上蔡主席選後並未接受採訪，為免混淆錯引，特此說明。,shared_story,http://www.dpp.org.tw/news_content.php?sn=8780,關於「韓聯社」刊載「獨家專訪」新聞 民進黨說明,115,1585,23,2016/01/29 10:24:39,
1_1,comment,劉明財(194451174225023), ,2016/01/19 19:21:00, ,10153180273481058_10153180274306058,, , , , ,1, , ,
1_2,comment,劉明財(194451174225023), ,2016/01/19 19:21:06, ,10153180273481058_10153180274376058,, , , , ,0, , ,
1_3,comment,劉明財(194451174225023), ,2016/01/19 19:21:19, ,10153180273481058_10153180274546058,, , , , ,0, , ,
1_4,comment,劉明財(194451174225023), ,2016/01/19 19:21:30, ,10153180273481058_10153180274651058,, , , , 

也用 pandas 看一下

In [2]:
import pandas as pd

In [3]:
data=pd.read_csv('demo_test.csv',header=0,encoding="utf8")

In [4]:
data.head()

,流水號,大類別,From,post_url,created_time,post_type,postID,post_message,status_type,link,轉貼標題(name),留言數,按讚數,分享數,update_time,Unnamed: 15
0,1,post,民主進步黨(150336441057),http://www.facebook.com/150336441057_101531802...,2016/01/19 19:20:06,link,150336441057_10153180273481058,關於「韓聯社」刊載「獨家專訪」新聞 民進黨說明 針對韓國媒體「韓聯社」於今（19）...,shared_story,http://www.dpp.org.tw/news_content.php?sn=8780,關於「韓聯社」刊載「獨家專訪」新聞 民進黨說明,115,1585,23,2016/01/29 10:24:39,NaN
1,1_1,comment,劉明財(194451174225023),,2016/01/19 19:21:00,,10153180273481058_10153180274306058,NaN,,,,,1,,,NaN
2,1_2,comment,劉明財(194451174225023),,2016/01/19 19:21:06,,10153180273481058_10153180274376058,NaN,,,,,0,,,NaN
3,1_3,comment,劉明財(194451174225023),,2016/01/19 19:21:19,,10153180273481058_10153180274546058,NaN,,,,,0,,,NaN
4,1_4,comment,劉明財(194451174225023),,2016/01/19 19:21:30,,10153180273481058_10153180274651058,NaN,,,,,0,,,NaN


## 資料匯入 elasticsearch

In [5]:
import csv
import os
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch.client import IndicesClient

In [6]:
es = Elasticsearch()
es_index=IndicesClient(es)

## 先建好 index
並使用 smartcn 分詞器

In [20]:
es.indices.create(
    index="platform",
    body={
      'settings': {
        'number_of_shards': 5,
        'number_of_replicas': 1,
        'analysis': {
          "analyzer":{"default":{"type": "smartcn"}}
          }
        },
      'mappings':{
         'facebook':{
            'properties':{
                u'留言數':{'type': 'integer'}
                }
            }   
        }  
      },
    ignore=400
)

{'error': {'index': 'platform',
  'reason': 'already exists',
  'root_cause': [{'index': 'platform',
    'reason': 'already exists',
    'type': 'index_already_exists_exception'}],
  'type': 'index_already_exists_exception'},
 'status': 400}

### 可用 head plugin 看創建狀態
http://localhost:9200/_plugin/head/

In [27]:
def CSVimportES(indexName,typeName,fileName):
    if os.path.isfile(fileName) == False:
        print ("The file dose not exist.")
    if es_index.exists(indexName) == False:
        es.indices.create(
            index=indexName,
            body={'settings': {
                    'number_of_shards': 5,'number_of_replicas': 1,
                    'analysis': {"analyzer":{"default":{"type": "smartcn"}}}},
                  'mappings': {
                    typeName: {
                        "date_detection": False,
                        "properties":{
                            u'留言數':{'type': 'integer'}
                        }
                    }
        
                }
                  #'mappings':{"_all":{"ignore_malformed": true}}
            },
            ignore=400)
    datas=[]
    with open('demo_test.csv','r+') as p_file:
        raw_data=csv.DictReader(p_file)
        for item in raw_data:
            datas.append({"_index":indexName,"_type":typeName,"_source":item})
    helpers.bulk(es,datas,chunk_size=100)

In [28]:
CSVimportES("platform","facebook","demo_test.csv") #bulk 匯入

## 資料搜索
<h3 style="color:#3030FF">設定分詞器</h3>(或者建 index 就設)

In [10]:
import requests

In [11]:
#先關閉
res=requests.post("http://localhost:9200/testdatabulk/_close")

In [12]:
set_ana='{"analysis":{"analyzer":{"default":{"type": "smartcn"}}}}'

In [13]:
res=requests.put("http://localhost:9200/testdatabulk/_setting",data=set_ana)

In [14]:
#再開啟
res=requests.post("http://localhost:9200/testdatabulk/_open")

## 連上 elasticsearch
<p style="color:#3030FF">elasticsearch 搜索</p>

In [15]:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es_index=IndicesClient(es)

## 看到所有欄位 還有狀態


In [18]:
es_index.get_mapping(index="platform",doc_type="facebook")

{'platform': {'mappings': {'facebook': {'properties': {'': {'type': 'string'},
     'From': {'type': 'string'},
     'created_time': {'format': 'yyyy/MM/dd HH:mm:ss||yyyy/MM/dd||epoch_millis',
      'type': 'date'},
     'link': {'type': 'string'},
     'postID': {'type': 'string'},
     'post_message': {'type': 'string'},
     'post_type': {'type': 'string'},
     'post_url': {'type': 'string'},
     'status_type': {'type': 'string'},
     'update_time': {'type': 'string'},
     '分享數': {'type': 'string'},
     '大類別': {'type': 'string'},
     '按讚數': {'type': 'string'},
     '流水號': {'type': 'string'},
     '留言數': {'type': 'string'},
     '轉貼標題(name)': {'type': 'string'}}}}}}

## 搜尋後顯示前幾筆
size 預設為 10
<div>"_all" 搜尋所有欄位</div>
<div> term 系列沒成功</div>

## 把檔案清空

In [21]:
es.indices.delete(index="platform",ignore=[400,404])

{'acknowledged': True}

In [67]:
result= es.search(index="platform",doc_type="facebook",body={"size":3,"query":{"match":{u"post_message":u"支持"}}})

一些結果的概況
用 aggregations 也可以用 aggs 
## 看搜尋的結果數量

In [19]:
es.count(index="platform",doc_type="facebook",body={"query":{"match":{u"post_message":u"支持"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5}, 'count': 23}

In [86]:
result1=es.search(index="platform",doc_type="facebook",body={"query":{"match":{u"post_message":u"支持"}}})

In [87]:
result1

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVKvjyV5AFzOQVlaeGyd',
    '_index': 'platform',
    '_score': 0.10980196,
    '_source': {'': '',
     'From': '民主進步黨(150336441057)',
     'created_time': '2016/01/19 15:28:14',
     'link': 'http://www.dpp.org.tw/news_content.php?sn=8779',
     'postID': '150336441057_10153180078106058',
     'post_message': '蔡英文、陳建仁全國各縣市感恩茶會巡迴謝票之旅 21日高雄出發為感謝各地輔選幹部、百工百業及志工在選戰期間的力挺與支持，總統當選人蔡英文及副總統當選人陳建仁預定從21日本周四啟動全台謝票，首日規劃走訪高雄市、屏東縣、台南市，周五宜蘭縣，23、24、26、28、29日將陸續安排其他縣市進行謝票。#感謝你們',
     'post_type': 'link',
     'post_url': 'http://www.facebook.com/150336441057_10153180078106058',
     'status_type': 'shared_story',
     'update_time': '2016/01/23 12:48:06',
     '分享數': '102',
     '大類別': 'post',
     '按讚數': '4304',
     '流水號': '2',
     '留言數': '147',
     '轉貼標題(name)': '蔡英文、陳建仁全國各縣市感恩茶會巡迴謝票之旅 21日高雄出發'},
    '_type': 'facebook'}],
  'max_score': 0.10980196,
  'total': 1},
 'timed_out': False,
 'took': 3}

In [92]:
len(result1['hits']['hits'])

1

## 搜尋 and

In [75]:
es.search(index="platform",body={"query":{"match_phrase":{"_all":"支持"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVKmNZJ4AFzOQVladWOo',
    '_index': 'platform',
    '_score': 2.136736,
    '_source': {'': '',
     'From': '高天(145823499114383)',
     'created_time': '2016/01/20 15:34:45',
     'link': ' ',
     'postID': '10153180078106058_10153181628506058',
     'post_message': '支持民?党，支持蔡英文。',
     'post_type': ' ',
     'post_url': ' ',
     'status_type': ' ',
     'update_time': ' ',
     '分享數': ' ',
     '大類別': 'comment',
     '按讚數': '0',
     '流水號': '2_110',
     '留言數': ' ',
     '轉貼標題(name)': ' '},
    '_type': 'facebook'},
   {'_id': 'AVKgI87MAFzOQVlac-iL',
    '_index': 'platform',
    '_score': 2.047713,
    '_source': {'': '',
     'From': '高天(145823499114383)',
     'created_time': '2016/01/20 15:34:45',
     'link': ' ',
     'postID': '10153180078106058_10153181628506058',
     'post_message': '支持民?党，支持蔡英文。',
     'post_type': ' ',
     'post_url': ' ',
     'status_type': ' ',
     'update_time': '

In [42]:
need_save = es.search(index='testdatabulk',doc_type='testdata', body={"query":{"match":{"_all":u"肉鬆"}},"sort":{"count":{"order":"asc"}}})

## 存成 csv 檔
存讀檔用法 http://www.tutorialspoint.com/python/python_files_io.htm 

In [16]:
import json

dumps 是變成 json format 的 string </p> 
dump 是存成 string file </p>
http://edumaven.com/python-programming/json-dump

In [20]:
need_save = json.dumps(need_save,ensure_ascii=False).encode('utf8')

In [21]:
h = open("test_demo.csv","wb+")
h.write(need_save)
h.close()

存成 json 檔

In [104]:
g = open('save_the_file.json','w+')
json.dump(need_save,g)
g.close()

In [109]:
#打開看存檔
g = open('save_the_file.json','r+')
y = json.load(g)
g.close()

In [110]:
print y

{"hits": {"hits": [{"sort": ["1"], "_type": "testdata", "_source": {"count": "1", "Pro_name": "肉鬆", "Pro_ctgry": "肉鬆/酥", "Adderres": "台東市光明路158號", "﻿No": "28", "Stock": "0.015", "Conty": "台東市", "Producder": "叢林之屋"}, "_score": null, "_index": "testdatabulk", "_id": "AVKM8fnQqiB9wr87bq_J"}, {"sort": ["1"], "_type": "testdata", "_source": {"count": "1", "Pro_name": "肉圓", "Pro_ctgry": "餅皮/麵皮", "Adderres": "北斗鎮中192號", "﻿No": "36", "Stock": "0.075", "Conty": "彰化縣", "Producder": "北斗肉圓生"}, "_score": null, "_index": "testdatabulk", "_id": "AVKM8fnQqiB9wr87bq_R"}, {"sort": ["1"], "_type": "testdata", "_source": {"count": "1", "Pro_name": "肉鬆", "Pro_ctgry": "肉鬆/酥", "Adderres": "台東市光明路158號", "﻿No": "28", "Stock": "0.015", "Conty": "台東市", "Producder": "叢林之屋"}, "_score": null, "_index": "testdatabulk", "_id": "AVKcSQ3oAFzOQVlacxQV"}, {"sort": ["1"], "_type": "testdata", "_source": {"count": "1", "Pro_name": "特純肉鬆、特級肉脯", "Pro_ctgry": "肉鬆/酥", "Adderres": "長春路15號", "﻿No": "26", "Stock": "2.25", "Conty"

In [ ]:
es.get(index='sw', doc_type='people', id=5)

In [ ]:
es.search(index="sw", body={"query": {"match": {'name':'Darth Vader'}}})

In [14]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'sw',
    u'_score': 1.0,
    u'_source': {u'birth_year': u'19BBY',
     u'created': u'2014-12-09T13:50:51.644000Z',
     u'edited': u'2014-12-20T21:17:56.891000Z',
     u'eye_color': u'blue',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/',
      u'http://swapi.co/api/films/7/'],
     u'gender': u'male',
     u'hair_color': u'blond',
     u'height': u'172',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'77',
     u'name': u'Luke Skywalker',
     u'skin_color': u'fair',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/12/',
      u'http://swapi.co/api/starships/22/'],
     u'url': u'http://swapi.co/api/people/1/',
     u'vehicles': [u'http://swapi.co/api/vehicles/14/',
      u'http: